# RAG_Flask_Ollama

In [ ]:
#Import the necessary libraries

import numpy as np
import sys
import langchain
import langchain_community
import langchain_core
import langchain_openai
import sentence_transformers
import pypdf
import dotenv
import importlib.metadata

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

print('Version information')

print('python: {}'.format(sys.version))
print('numpy: {}'.format(np.__version__))
print('langchain: {}'.format(langchain.__version__))
print('langchain_community: {}'.format(langchain_community.__version__))
print('sentence_transformers: {}'.format(sentence_transformers.__version__))
print('pypdf: {}'.format(pypdf.__version__))
print('langchain_core: {}'.format(langchain_core.__version__))
print('langchain_openai: {}'.format(importlib.metadata.version("langchain-openai")))
print('python-dotenv: {}'.format(importlib.metadata.version("python-dotenv")))
print('ollama: {}'.format(importlib.metadata.version("ollama")))

## Retrieval Augmented Generation (RAG)

State-of-the-art Large Language Models (LLM's) are trained on an enormous corpus of textual data (taken from webpages, articles, books etc.), and they store a wide range of general knowledge in their parameters. While they are able to perform well on tasks that require general knowledge, they tend to struggle on tasks that require information that wasn't present in the training data. For instance, LLM's may struggle on tasks that require knowledge of domain-specific information, or even up-to-date information.

It is very important to overcome this problem, because it is undesirable to get a non-answer from the LLM, and potentially even dangerous if the LLM begins to hallucinate (i.e ramble on with an answer that seems believable but is factually inaccurate). Therefore, it is essential to bridge the gap between the LLM's general knowledge, and other domain-specific or up-to-date information in order to help the LLM generate responses that are contextual and factually accurate, while reducing the chances of hallucinations.

There are two effective ways of accomplishing this:
1. <strong>Fine-tuning the LLM on the domain-specific/proprietary/new data</strong>:
    <ul>
        <li>By fine-tuning the model, it can be made suitable for the task.</li>
        <li>However, this comes with some limitations. It is compute-intensive, expensive and not agile (it's not realistic to fine-tune an LLM with the new data coming in everyday)</li>
    </ul>
2. <strong>Retrieval Augmented Generation (RAG)</strong>:
    <ul>
        <li><a href='https://arxiv.org/abs/2005.11401'>This technique</a> provides the LLM with contextual information from an external knowledge source that can be updated more easily.</li>
        <li>It allows the LLM to generate more contextual, and factually accurate responses by allowing it to dynamically access information from an external knowledge source.</li>
    </ul>

<center><img src="data/images/rag-architecture.png" alt="drawing" width="700" align='center'></center>
<center>Basic RAG Architecture</center>


The RAG pipeline consists of the following steps:
<ol>
    <li><strong>Retrieval</strong>: The user's query is used to retrieve the relevant contextual information from the external knowledge source. The external knowledge source is a vector store that contains the embeddings of the documents that contain the proprietary/domain-specific data. The user query is embedded into the same vector space as these documents, and a similarity search is performed in this embedding space to retrieve the documents that are most similar to the user's query. These retrieved documents make up the context that the LLM will consider in order to produce factually accurate responses. </li>
    <li><strong>Augmentation</strong>: The user's query is augmented with the retrieved contextual information to form the prompt. The prompt will also usually include instructions to the LLM for performing the task. There is an entire sub-field known as Prompt Engineering, that is dedicated to fine-tuning the prompt so as to get the best possible response from the LLM, and you will get some experience with it in <strong>7.2</strong> while implementing the RAG Chain. </li>
    <li><strong>Generation</strong>: The constructed prompt, including the instructions to the LLM, the user query and the retrieved context, is fed to the LLM to generate a response that is comprehensible and factually correct.</li>
</ol>

## 1) Implementing the Retriever

In the first step, the Retriever for the RAG Chain will be implemented. The documents serving as the basis of the external knowledge source will be split into smaller chunks to be used with the vector database. Lastly, the Retriever object will be created to be used with the RAG Chain to retrieve relevant document chunks as additional contextual information.

### 1.1) Loading and Pre-processing data: Document Loaders and Text Splitters

Langchain provides <a href='https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/'>several classes</a> to load data into Document objects. There are classes to load data from HTML files, PDF files, JSON files, CSV files, file directories etc. Here, Langchain's <a href='https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/#pypdf-directory'>PyPDFDirectoryLoader</a> will be used to load PDF's from a directory. A Document object is a dictionary that stores the text and metadata about each document.

Once the data has been loaded into Document(s), it's common to split the documents into smaller chunks. This is done because when a user inputs a query to the system, the retriever will return the most relevant documents, which will be augmented to the prompt that is sent to the LLM. There are limits to the length of this prompt, since it must fit into the LLM's context window. Therefore, it's common to split documents into smaller chunks, so that only the retrieved relevant chunks are inserted into the prompt. Langchain offers several <a href='https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/'>TextSplitters</a> to perform the document splitting. RecursiveCharacterTextSplitter, which is a way to split a document into several chunks in a manner such that related pieces of text are kept together in a chunk, will be used here.